# Pré-processamento de dados
## Importando bibliotecas

In [1]:
from itertools import product
import string

import pandas as pd
import numpy as np
import spacy
import nltk
from nltk.corpus import stopwords
from spacy.lang.pt.examples import sentences
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import ShuffleSplit, cross_validate

from src.data import preprocessing


nltk.download('rslp')
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/madson/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package rslp to /home/madson/nltk_data...
[nltk_data]   Package rslp is already up-to-date!
[nltk_data] Downloading package punkt to /home/madson/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/madson/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## Carregamento dos Dados

In [2]:
output_path = "../data/interim/news.csv"
df_corpus = pd.read_csv(output_path)
df_corpus.head(5)

,text,label
0,JBS pagará R$ 650 mil de indenização por demis...,true
1,Motorista envolvido em morte de comissário já ...,true
2,Em propaganda partidária veiculada anteontem ...,true
3,PGR pede ao STF que reverta decisão do Legisla...,true
4,Como o Ocidente compreende mal a China. Apesar...,true


In [3]:
df_corpus = (
    df_corpus
    .assign(
        clean_text = preprocessing.lower_case(df_corpus['text'])
    )
)
df_corpus.head(5)

,text,label,clean_text
0,JBS pagará R$ 650 mil de indenização por demis...,true,jbs pagará r$ 650 mil de indenização por demis...
1,Motorista envolvido em morte de comissário já ...,true,motorista envolvido em morte de comissário já ...
2,Em propaganda partidária veiculada anteontem ...,true,em propaganda partidária veiculada anteontem ...
3,PGR pede ao STF que reverta decisão do Legisla...,true,pgr pede ao stf que reverta decisão do legisla...
4,Como o Ocidente compreende mal a China. Apesar...,true,como o ocidente compreende mal a china. apesar...


## Removendo termos específicos

### Removendo datas (Formato dd/mm/YYYY)

In [4]:
df_corpus = (
    df_corpus
    .assign(
        clean_text = preprocessing.remove_date(df_corpus.clean_text)
    )
)
df_corpus

Modificando 103 linhas contendo este padrão.


,text,label,clean_text
0,JBS pagará R$ 650 mil de indenização por demis...,true,jbs pagará r$ 650 mil de indenização por demis...
1,Motorista envolvido em morte de comissário já ...,true,motorista envolvido em morte de comissário já ...
2,Em propaganda partidária veiculada anteontem ...,true,em propaganda partidária veiculada anteontem ...
3,PGR pede ao STF que reverta decisão do Legisla...,true,pgr pede ao stf que reverta decisão do legisla...
4,Como o Ocidente compreende mal a China. Apesar...,true,como o ocidente compreende mal a china. apesar...
...,...,...,...
7195,Coreia anuncia novo teste nuclear. Diplomata d...,fake,coreia anuncia novo teste nuclear. diplomata d...
7196,Temer vai punir Sergio Reis e Tiririca por tra...,fake,temer vai punir sergio reis e tiririca por tra...
7197,Wikileaks diz que Temer era espião dos EUA. . ...,fake,wikileaks diz que temer era espião dos eua. . ...
7198,Site divulga que viagem de Lula à Europa seria...,fake,site divulga que viagem de lula à europa seria...


### Removendo datas (Formato dd de mês de YYYY)

In [5]:
# Remove datas no formato dd de mês de YYYY
df_corpus = (
    df_corpus
    .assign(
        clean_text = preprocessing.remove_date(df_corpus.clean_text, style='diamesextensoano')
    )
)
df_corpus

Modificando 643 linhas contendo este padrão.


,text,label,clean_text
0,JBS pagará R$ 650 mil de indenização por demis...,true,jbs pagará r$ 650 mil de indenização por demis...
1,Motorista envolvido em morte de comissário já ...,true,motorista envolvido em morte de comissário já ...
2,Em propaganda partidária veiculada anteontem ...,true,em propaganda partidária veiculada anteontem ...
3,PGR pede ao STF que reverta decisão do Legisla...,true,pgr pede ao stf que reverta decisão do legisla...
4,Como o Ocidente compreende mal a China. Apesar...,true,como o ocidente compreende mal a china. apesar...
...,...,...,...
7195,Coreia anuncia novo teste nuclear. Diplomata d...,fake,coreia anuncia novo teste nuclear. diplomata d...
7196,Temer vai punir Sergio Reis e Tiririca por tra...,fake,temer vai punir sergio reis e tiririca por tra...
7197,Wikileaks diz que Temer era espião dos EUA. . ...,fake,wikileaks diz que temer era espião dos eua. . ...
7198,Site divulga que viagem de Lula à Europa seria...,fake,site divulga que viagem de lula à europa seria...


### Dias da semana

In [6]:
# remove dias da semana
df_corpus['clean_text'] = df_corpus['clean_text'].apply(lambda c: preprocessing.remove_weekday(c))
df_corpus.sample(10)

,text,label,clean_text
3191,Nada falta ao depoimento mais recente de Marc...,true,nada falta ao depoimento mais recente de marc...
4004,Vazou a carta pessoal que Temer enviou para Di...,fake,vazou a carta pessoal que temer enviou para di...
5327,"Durante programa do SBT, Mara Maravilha faz co...",fake,"durante programa do sbt, mara maravilha faz co..."
1391,Zuma diz que não vai renunciar e que pressão d...,true,zuma diz que não vai renunciar e que pressão d...
6868,Homicídio qualificado: Delegado pede prisão de...,fake,homicídio qualificado: delegado pede prisão de...
5314,"Eduardo Cunha, o novo aliado do PT, legisla ...",fake,"eduardo cunha, o novo aliado do pt, legisla ..."
1159,O lobista da Odebrecht no Congresso Claudio M...,true,o lobista da odebrecht no congresso claudio m...
970,"Para juristas, atuação das Forças Armadas em p...",true,"para juristas, atuação das forças armadas em p..."
1767,Candidato José Ricardo participa de entrevista...,true,candidato josé ricardo participa de entrevista...
5029,"Gilmar Mendes: ""Nomeação de Lula é uma barbári...",fake,"gilmar mendes: ""nomeação de lula é uma barbári..."


### Removendo "\n"

In [7]:
df_corpus['clean_text'] = preprocessing.remove_linebreaks(df_corpus.clean_text)
df_corpus

Modificando 577 linhas contendo este padrão.


,text,label,clean_text
0,JBS pagará R$ 650 mil de indenização por demis...,true,jbs pagará r$ 650 mil de indenização por demis...
1,Motorista envolvido em morte de comissário já ...,true,motorista envolvido em morte de comissário já ...
2,Em propaganda partidária veiculada anteontem ...,true,em propaganda partidária veiculada anteontem ...
3,PGR pede ao STF que reverta decisão do Legisla...,true,pgr pede ao stf que reverta decisão do legisla...
4,Como o Ocidente compreende mal a China. Apesar...,true,como o ocidente compreende mal a china. apesar...
...,...,...,...
7195,Coreia anuncia novo teste nuclear. Diplomata d...,fake,coreia anuncia novo teste nuclear. diplomata d...
7196,Temer vai punir Sergio Reis e Tiririca por tra...,fake,temer vai punir sergio reis e tiririca por tra...
7197,Wikileaks diz que Temer era espião dos EUA. . ...,fake,wikileaks diz que temer era espião dos eua. . ...
7198,Site divulga que viagem de Lula à Europa seria...,fake,site divulga que viagem de lula à europa seria...


### Removendo "\t"

In [8]:
df_corpus['clean_text'] = preprocessing.remove_tabs(df_corpus.clean_text)
df_corpus

Modificando 3 linhas contendo este padrão.


,text,label,clean_text
0,JBS pagará R$ 650 mil de indenização por demis...,true,jbs pagará r$ 650 mil de indenização por demis...
1,Motorista envolvido em morte de comissário já ...,true,motorista envolvido em morte de comissário já ...
2,Em propaganda partidária veiculada anteontem ...,true,em propaganda partidária veiculada anteontem ...
3,PGR pede ao STF que reverta decisão do Legisla...,true,pgr pede ao stf que reverta decisão do legisla...
4,Como o Ocidente compreende mal a China. Apesar...,true,como o ocidente compreende mal a china. apesar...
...,...,...,...
7195,Coreia anuncia novo teste nuclear. Diplomata d...,fake,coreia anuncia novo teste nuclear. diplomata d...
7196,Temer vai punir Sergio Reis e Tiririca por tra...,fake,temer vai punir sergio reis e tiririca por tra...
7197,Wikileaks diz que Temer era espião dos EUA. . ...,fake,wikileaks diz que temer era espião dos eua. . ...
7198,Site divulga que viagem de Lula à Europa seria...,fake,site divulga que viagem de lula à europa seria...


### Removendo símbolos

In [9]:
df_corpus['clean_text'] = preprocessing.symbol_remove(df_corpus.clean_text)
df_corpus

Modificando 51820 linhas contendo este padrão.


,text,label,clean_text
0,JBS pagará R$ 650 mil de indenização por demis...,true,jbs pagará r 650 mil de indenização por demiss...
1,Motorista envolvido em morte de comissário já ...,true,motorista envolvido em morte de comissário já ...
2,Em propaganda partidária veiculada anteontem ...,true,em propaganda partidária veiculada anteontem ...
3,PGR pede ao STF que reverta decisão do Legisla...,true,pgr pede ao stf que reverta decisão do legisla...
4,Como o Ocidente compreende mal a China. Apesar...,true,como o ocidente compreende mal a china apesar ...
...,...,...,...
7195,Coreia anuncia novo teste nuclear. Diplomata d...,fake,coreia anuncia novo teste nuclear diplomata di...
7196,Temer vai punir Sergio Reis e Tiririca por tra...,fake,temer vai punir sergio reis e tiririca por tra...
7197,Wikileaks diz que Temer era espião dos EUA. . ...,fake,wikileaks diz que temer era espião dos eua o...
7198,Site divulga que viagem de Lula à Europa seria...,fake,site divulga que viagem de lula à europa seria...


## Tokenização

In [10]:
df_corpus['tokens'] = preprocessing.tokenize(df_corpus.clean_text)
df_corpus

,text,label,clean_text,tokens
0,JBS pagará R$ 650 mil de indenização por demis...,true,jbs pagará r 650 mil de indenização por demiss...,"[jbs, pagará, r, 650, mil, de, indenização, po..."
1,Motorista envolvido em morte de comissário já ...,true,motorista envolvido em morte de comissário já ...,"[motorista, envolvido, em, morte, de, comissár..."
2,Em propaganda partidária veiculada anteontem ...,true,em propaganda partidária veiculada anteontem ...,"[em, propaganda, partidária, veiculada, anteon..."
3,PGR pede ao STF que reverta decisão do Legisla...,true,pgr pede ao stf que reverta decisão do legisla...,"[pgr, pede, ao, stf, que, reverta, decisão, do..."
4,Como o Ocidente compreende mal a China. Apesar...,true,como o ocidente compreende mal a china apesar ...,"[como, o, ocidente, compreende, mal, a, china,..."
...,...,...,...,...
7195,Coreia anuncia novo teste nuclear. Diplomata d...,fake,coreia anuncia novo teste nuclear diplomata di...,"[coreia, anuncia, novo, teste, nuclear, diplom..."
7196,Temer vai punir Sergio Reis e Tiririca por tra...,fake,temer vai punir sergio reis e tiririca por tra...,"[temer, vai, punir, sergio, reis, e, tiririca,..."
7197,Wikileaks diz que Temer era espião dos EUA. . ...,fake,wikileaks diz que temer era espião dos eua o...,"[wikileaks, diz, que, temer, era, espião, dos,..."
7198,Site divulga que viagem de Lula à Europa seria...,fake,site divulga que viagem de lula à europa seria...,"[site, divulga, que, viagem, de, lula, à, euro..."


## Removendo Stopwords

In [11]:
stop_words = stopwords.words('portuguese')
df_corpus = (
    df_corpus
    .assign(
        no_stopwords = df_corpus.tokens.apply(lambda x: tuple(filter(lambda y: y not in stop_words, x)))
    )
)

## Lematização

In [12]:
# nlp = spacy.load('pt_core_news_sm')
# df_corpus = (
#     df_corpus
#     .assign(
#         lemmatization_final_texts = df_corpus.no_stopwords.apply(lambda x: nlp(" ".join(x)))
#     )
# )

In [13]:
df_corpus['lemmatization_final_texts'] = preprocessing.lemmatization(df_corpus['no_stopwords'])
df_corpus

## Stemming

In [14]:
df_corpus['stemming_final_texts'] = preprocessing.stemming(df_corpus['no_stopwords'])
df_corpus

,text,label,clean_text,tokens,no_stopwords,lemmatization_final_texts,stemming_final_texts
0,JBS pagará R$ 650 mil de indenização por demis...,true,jbs pagará r 650 mil de indenização por demiss...,"[jbs, pagará, r, 650, mil, de, indenização, po...","(jbs, pagará, r, 650, mil, indenização, demiss...","(jbs, pagará, r, 650, mil, indenização, demiss...","[jb, pag, r, 650, mil, inden, demiss, mass, mt..."
1,Motorista envolvido em morte de comissário já ...,true,motorista envolvido em morte de comissário já ...,"[motorista, envolvido, em, morte, de, comissár...","(motorista, envolvido, morte, comissário, proc...","(motorista, envolvido, morte, comissário, proc...","[motor, envolv, mort, comiss, process, prefeit..."
2,Em propaganda partidária veiculada anteontem ...,true,em propaganda partidária veiculada anteontem ...,"[em, propaganda, partidária, veiculada, anteon...","(propaganda, partidária, veiculada, anteontem,...","(propaganda, partidária, veiculada, anteontem,...","[propagand, partidár, veicul, anteont, televis..."
3,PGR pede ao STF que reverta decisão do Legisla...,true,pgr pede ao stf que reverta decisão do legisla...,"[pgr, pede, ao, stf, que, reverta, decisão, do...","(pgr, pede, stf, reverta, decisão, legislativo...","(pgr, pede, stf, reverta, decisão, legislativo...","[pgr, ped, stf, revert, decis, legisl, potigu,..."
4,Como o Ocidente compreende mal a China. Apesar...,true,como o ocidente compreende mal a china apesar ...,"[como, o, ocidente, compreende, mal, a, china,...","(ocidente, compreende, mal, china, apesar, apo...","(ocidente, compreende, mal, china, apesar, apo...","[ocid, compreend, mal, chin, apes, apost, líd,..."
...,...,...,...,...,...,...,...
7195,Coreia anuncia novo teste nuclear. Diplomata d...,fake,coreia anuncia novo teste nuclear diplomata di...,"[coreia, anuncia, novo, teste, nuclear, diplom...","(coreia, anuncia, novo, teste, nuclear, diplom...","(coreia, anuncia, novo, teste, nuclear, diplom...","[core, anunc, nov, test, nucle, diplomat, diz,..."
7196,Temer vai punir Sergio Reis e Tiririca por tra...,fake,temer vai punir sergio reis e tiririca por tra...,"[temer, vai, punir, sergio, reis, e, tiririca,...","(temer, vai, punir, sergio, reis, tiririca, tr...","(temer, vai, punir, sergio, reis, tiririca, tr...","[tem, vai, pun, sergi, reil, tirir, tra, sergi..."
7197,Wikileaks diz que Temer era espião dos EUA. . ...,fake,wikileaks diz que temer era espião dos eua o...,"[wikileaks, diz, que, temer, era, espião, dos,...","(wikileaks, diz, temer, espião, eua, wikileaks...","(wikileaks, diz, temer, espião, eua, wikileaks...","[wikileak, diz, tem, espiã, eua, wikileak, por..."
7198,Site divulga que viagem de Lula à Europa seria...,fake,site divulga que viagem de lula à europa seria...,"[site, divulga, que, viagem, de, lula, à, euro...","(site, divulga, viagem, lula, europa, tratamen...","(site, divulga, viagem, lula, europa, tratamen...","[sit, divulg, viag, lul, europ, trat, possível..."


## Salvando dados pré-processados

In [15]:
(
    df_corpus
    .drop(columns=["clean_text", "tokens", "no_stopwords"], axis=1)
    .to_csv("../data/processed/textos_pre-processado.csv", index=False)
)